# Stochastic Variational Method with RL algorithms

In [1]:
import numpy as np
import gym
import torch
import subprocess
import os
import pickle

## Expoloring environment

In [2]:
env = gym.make('svm_env:svmEnv-v2', n_pairs = 3, n_basis = 50, file_sigmas ="./svmCodeSVD/sigmas.dat" )

print('### Env Name ######', env.unwrapped.spec.id)

obs_space = env.observation_space

print('###### Observation space ####### \n', obs_space)

state_size = env.observation_space.shape[-1]

print('###### Size of observation space ####### \n', state_size)

act_space = env.action_space.shape

print('###### Action space ####### \n', act_space)

act_size = env.action_space.shape[0]*env.action_space.shape[-1]

print('###### Number of actions ####### \n', act_size)

state = env.reset()

print('##### State after reset ###### \n', state)

print('##### File where will be stored sigmas \n', env.file_sigmas)



### Env Name ###### svmEnv-v2
###### Observation space ####### 
 Box(-inf, inf, (1,), float32)
###### Size of observation space ####### 
 1
###### Action space ####### 
 (50, 3)
###### Number of actions ####### 
 150
*****CALL RESET******
Action chosen at reset:  [0.]
##### State after reset ###### 
 [0.]
##### File where will be stored sigmas 
 ./svmCodeSVD/sigmas.dat


# Your codes `DDPG` and `PPO`

## Functions for saving and clean

In [3]:
## Save all rewards, energies and princip dims in files during training
def create_info_h5(agent, env):
    # Check if file exist and creat it
    i = 0
    while os.path.exists(f'run_{i}.hdf5'):
        i += 1
    dataFile = h5py.File(f'run_{i}.hdf5', 'a')
    
    # Create dataset to store info in hdf5 file
    info = {'alg':agent.name, 'env':env.unwrapped.spec.id}
    st = h5py.string_dtype(encoding='utf-8')
    dataFile.create_dataset('info', dtype=st)
    for k in info.keys():
        dataFile['info'].attrs[k] = info[k]

    # Create dataset to store hyperparams of the model in hdf5 file
    hyperparams = {'batch_size':agent.batch_size, 'bootstrap_size':agent.bootstrap_size \
                   , 'gamma':agent.gamma, 'tau':agent.tau,'lr_critic':agent.lr_critic \
                  , 'lr_actor':agent.lr_actor, 'update_every':agent.update_every \
                   , 'transfer_every':agent.transfer_every, 'num_update':agent.num_update \
                  , 'add_noise_every':agent.add_noise_every}
    dataFile.create_dataset('hyperparams', dtype='f')
    for k in hyperparams.keys():
        dataFile['hyperparams'].attrs[k] = hyperparams[k]
    
    # Create group for rewards, energies, princip dims, actor and critic model
    dataFile.create_group('sigmas')
    dataFile.create_group('rewards')
    dataFile.create_group('energies')
    dataFile.create_group('princip_dims')
    dataFile.create_group('full_dims')   
    dataFile.create_group('actor_models')
    dataFile.create_group('critic_models')
    
    # Close and return data file name
    dataFile_name = dataFile.filename
    dataFile.close()
    
    return dataFile_name

def save_all(dat_file_name, i_ep, sigmas_i_ep, rew_i_ep, en_i_ep, pri_dim_i_ep \
             , full_dim_i_ep, act_model_i_ep, cr_model_i_ep):
    # Open data file
    dat_file = h5py.File(dat_file_name, 'a')
    
    # Create datasets for rewards, energies, pri dim, full dim and store data in it 
    dat_file['sigmas'].create_dataset(f'sigmas_ep_{i_ep}', dtype='f', data=sigmas_i_ep)
    dat_file['rewards'].create_dataset(f'rew_ep_{i_ep}', dtype='f', data=rew_i_ep)
    dat_file['energies'].create_dataset(f'en_ep_{i_ep}', dtype='f', data=en_i_ep)
    dat_file['princip_dims'].create_dataset(f'pri_dim_ep_{i_ep}', dtype='i', data=pri_dim_i_ep)
    dat_file['full_dims'].create_dataset(f'full_dim_ep_{i_ep}', dtype='i', data=full_dim_i_ep)
    
    # Store in actor models group the network params at each ep
    actor_model = torch.load(act_model_i_ep)
    dat_file['actor_models'].create_dataset(f'act_mod_{i_ep}', dtype='f')
    for k in actor_model.keys():
        dat_file['actor_models'][f'act_mod_{i_ep}'].attrs.create(name=k,data=actor_model[k].cpu().data.numpy())
    
    # Store in actor models group the network params at each ep
    critic_model = torch.load(cr_model_i_ep)
    dat_file['critic_models'].create_dataset(f'cri_mod_{i_ep}', dtype='f')
    for k in critic_model.keys():
        dat_file['critic_models'][f'cri_mod_{i_ep}'].attrs.create(name=k,data=critic_model[k].cpu().data.numpy())
    
    # Close data file
    dat_file.close()
    
def rm_useless_file(actor_model_file, critic_model_file, file_sigmas):
    os.remove(actor_model_file)
    os.remove(critic_model_file)
    os.remove(file_sigmas)

In [4]:
## Save all rewards, energies and princip dims in files during episode training
def create_run_fold_and_info(agent, env):
    
    # Check if folder exist and creat it
    i = 0
    while os.path.exists(f'runs/run_{i}/'):
        i += 1
    name_dir = f'runs/run_{i}/'
    os.makedirs(name_dir)
    
    # Create info.p to store info in pickle file
    info = {'alg':agent.name, 'env':env.unwrapped.spec.id \
            , 'batch_size':agent.batch_size, 'bootstrap_size':agent.bootstrap_size \
            , 'gamma':agent.gamma, 'tau':agent.tau,'lr_critic':agent.lr_critic \
            , 'lr_actor':agent.lr_actor, 'update_every':agent.update_every \
            , 'transfer_every':agent.transfer_every, 'num_update':agent.num_update \
            , 'add_noise_every':agent.add_noise_every}
    
    pickle.dump(info, open(name_dir+'info.p', 'wb'))
    return name_dir
    
def save_all(name_run_dir, i_ep, sigmas_i_ep, rew_i_ep, en_i_ep, pri_dim_i_ep \
             , full_dim_i_ep, act_model_i_ep, cr_model_i_ep):
    
    pickle.dump(sigmas_i_ep, open(name_run_dir+f'sigmas_{i_ep}.p', 'wb'))
    pickle.dump(rew_i_ep, open(name_run_dir+f'rew_{i_ep}.p', 'wb'))
    pickle.dump(pri_dim_i_ep, open(name_run_dir+f'pri_dim_{i_ep}.p', 'wb'))
    pickle.dump(full_dim_i_ep, open(name_run_dir+f'full_dim_{i_ep}.p', 'wb'))
    pickle.dump(act_model_i_ep, open(name_run_dir+f'act_model_{i_ep}.p', 'wb'))
    pickle.dump(cr_model_i_ep, open(name_run_dir+f'cr_model_{i_ep}.p', 'wb'))
    
def rm_useless_file(actor_model_file, critic_model_file, file_sigmas):
    os.remove(actor_model_file)
    os.remove(critic_model_file)
    os.remove(file_sigmas)

## From my `ddpg_agent.py` code

In [5]:
from ddpg_agent import DDPG_agent
agent = DDPG_agent(state_size, act_size, seed = 0)

In [6]:
## Run ddpg algs   
def run_ddpg(max_t_step = 10, n_episodes=10):
    
    # Create h5 file and store info about alg and its hypereparams
    name_run_dir = create_run_fold_and_info(agent, env)
    
    for i_ep in range(n_episodes):
        state = env.reset()
        agent.reset()
        rew_i_ep = []
        en_i_ep = []
        pri_dim_i_ep = []
        full_dim_i_ep = []
        action_i_episode = []

        ## Training loop of each episode
        for t_step in range(max_t_step):
            action = agent.act(state)
            next_state, reward, done, info = env.step(action.reshape((env.n_basis,env.n_pairs)))
            agent.step(state, action, reward, next_state, done)
            state = next_state

            # Save rew, energies, princip dims, act and crit models
            action_i_episode.append(action.reshape((env.n_basis,env.n_pairs)))
            rew_i_ep.append(reward)
            en_i_ep.append(state[0])
            pri_dim_i_ep.append(env.princp_dim)
            full_dim_i_ep.append(env.full_dim)
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            if done:
                break
                
        ## Save data during training (to not lose the work done)
        save_all(name_run_dir=name_run_dir, i_ep=int(i_ep), sigmas_i_ep=action_i_episode \
                 , rew_i_ep=rew_i_ep, en_i_ep=en_i_ep, pri_dim_i_ep=pri_dim_i_ep \
                 , full_dim_i_ep=full_dim_i_ep, act_model_i_ep='checkpoint_actor.pth' \
                 , cr_model_i_ep='checkpoint_critic.pth')
        
        print('Episode {} ... Score: {:.3f}'.format(i_ep, np.sum(rew_i_ep)))

    rm_useless_file('checkpoint_actor.pth', 'checkpoint_critic.pth', env.file_sigmas)
    return name_run_dir

In [7]:
all_data = run_ddpg(10, 10)

*****CALL RESET******
Action chosen at reset:  [0.]
****CALL STEP****
Action chosen at step:  [[56.62079  74.54903  71.81472 ]
 [46.366604 60.00624  46.208206]
 [72.19338  64.05096  43.73308 ]
 [48.05089  35.115444 51.17658 ]
 [63.10283  58.69724  66.3265  ]
 [57.2352   49.77198  36.248352]
 [40.77845  61.163216 59.994396]
 [37.145775 61.90211  34.994244]
 [78.00186  53.567673 73.89515 ]
 [53.84555  77.4702   62.86515 ]
 [46.758446 37.703266 50.87429 ]
 [55.498627 79.23291  59.081318]
 [42.57069  51.847466 74.4142  ]
 [55.218266 67.95764  75.30478 ]
 [27.697033 78.35595  31.315296]
 [43.471325 50.295387 64.188805]
 [84.21481  42.8381   65.45694 ]
 [47.931084 58.694885 59.48698 ]
 [72.98885  71.01546  61.818535]
 [79.39163  59.21627  52.20235 ]
 [51.3312   41.98659  58.39671 ]
 [68.37305  51.81308  64.949615]
 [52.055717 37.52563  51.46722 ]
 [35.880962 65.45378  74.24716 ]
 [41.80757  60.92245  65.95514 ]
 [73.28565  72.23555  56.23385 ]
 [51.718163 54.84013  46.400253]
 [66.0576   59.

With this action the energy is:  -0.0934616
With this action the full dim is:  50  and princip dim is:  47
#### THE ACTION IS A GOOD ONE ####
**** THE AGENT STATE IS THE ENERGY **** -0.0934616
Set reward :  6.038733133529069
****CALL STEP****
Action chosen at step:  [[ 67.748825  79.58971   76.53144 ]
 [ 58.600243  75.84079   23.372826]
 [ 64.83693   57.99868   69.07976 ]
 [ 27.490805  27.792875  58.336563]
 [ 63.67158   52.627853  46.873528]
 [ 85.33781   23.590485  17.437279]
 [ 53.91177   45.675686  93.0444  ]
 [ 81.25935   89.3019    48.32096 ]
 [ 57.63809   49.745388  54.90922 ]
 [ 38.853973  49.514736  81.68819 ]
 [ 91.19113   67.06339   54.13704 ]
 [ 76.388626  65.701546  43.723   ]
 [ 63.982716  60.337086  84.41444 ]
 [ 73.44702   33.68642   81.78111 ]
 [ 49.323273  97.33129   40.453896]
 [ 70.22476   80.106735  85.906075]
 [ 81.39213   51.599445  71.04626 ]
 [ 66.56798   18.52155   27.139282]
 [ 57.469883  67.38969   48.699802]
 [ 56.261368  59.334637  48.910206]
 [ 51.706547 

With this action the energy is:  -0.127954
With this action the full dim is:  50  and princip dim is:  48
#### THE ACTION IS A GOOD ONE ####
**** THE AGENT STATE IS THE ENERGY **** -0.127954
Set reward :  8.43731612787925
****CALL STEP****
Action chosen at step:  [[ 47.801735   58.25182    76.5767   ]
 [ 68.39002    42.02959    28.365473 ]
 [ 51.855286   72.768394  105.16343  ]
 [ 37.13025    57.911922   72.10819  ]
 [ 68.4095     51.407017   52.15643  ]
 [ 97.549934   45.65104    25.673428 ]
 [ 51.584854   34.301987   96.03725  ]
 [ 34.56968    63.662743   47.811516 ]
 [ 44.81353    43.88756    71.5338   ]
 [  4.544899   74.16994    56.412064 ]
 [ 80.400604   68.26841    23.061874 ]
 [ 43.394653   56.33668    64.51883  ]
 [ 62.716232   63.26685    48.7362   ]
 [ 57.99383    50.376087   53.634327 ]
 [ 77.82807    76.59271    50.915146 ]
 [ 85.87834    40.995403   45.91805  ]
 [ 85.648315   56.48757    48.693493 ]
 [ 79.04066     1.         54.679176 ]
 [ 24.941689   41.570786   32.2909

With this action the energy is:  -0.0725703
With this action the full dim is:  50  and princip dim is:  47
#### THE ACTION IS A GOOD ONE ####
**** THE AGENT STATE IS THE ENERGY **** -0.0725703
Set reward :  4.585963337577848
****CALL STEP****
Action chosen at step:  [[ 55.111458  57.6152    60.326103]
 [ 33.0036    23.151745  39.776257]
 [ 35.67621   64.06865   50.972458]
 [ 53.03271   80.629105  53.218174]
 [ 57.78268   40.144096  67.46254 ]
 [ 37.65551   53.353916  49.11226 ]
 [ 83.08518   20.782162  60.735073]
 [ 32.79574   98.48084   55.367588]
 [ 54.156094  67.37548   96.26492 ]
 [ 29.565126  87.35093   79.96447 ]
 [ 48.413322  18.154594  75.94587 ]
 [ 59.06898   35.892754  60.703087]
 [ 28.65165   50.770226  27.80414 ]
 [ 19.752056  76.23792   59.34504 ]
 [ 30.630772  68.11322   55.052864]
 [ 37.902004  50.087505  75.08625 ]
 [ 48.306786  30.30736   64.62451 ]
 [ 54.668823  58.917618  63.0123  ]
 [ 54.15004   51.82509   74.53355 ]
 [ 64.78549   60.051414  57.709553]
 [ 45.187668 

With this action the energy is:  -0.0863902
With this action the full dim is:  50  and princip dim is:  49
#### THE ACTION IS A GOOD ONE ####
**** THE AGENT STATE IS THE ENERGY **** -0.0863902
Set reward :  5.546991770612952
****CALL STEP****
Action chosen at step:  [[18.921642  32.608566  41.315266 ]
 [21.857368  16.14795   47.83481  ]
 [15.033482  46.122223  50.85808  ]
 [62.663956  63.24831   71.51278  ]
 [54.31363   84.57776   75.653336 ]
 [40.145634  64.05784   38.98402  ]
 [55.442207   1.6973457 59.6579   ]
 [12.364811  83.92728   64.64378  ]
 [83.401825  74.69096   60.13481  ]
 [26.2741    87.9771    51.951782 ]
 [89.649475  54.87307   80.07233  ]
 [64.35091   39.020554  57.50876  ]
 [46.675705  66.62122   32.262947 ]
 [38.637016  83.21312   45.639107 ]
 [50.74434   35.872284  89.25559  ]
 [49.487774  56.140522  60.308357 ]
 [32.52081    1.        78.92904  ]
 [27.105957  54.061302  63.29855  ]
 [48.18659   88.28337   90.602905 ]
 [22.966576  66.185074  97.19718  ]
 [33.3488    

...

With this action the energy is:  -0.0354111
With this action the full dim is:  50  and princip dim is:  42
#### THE ACTION IS A GOOD ONE ####
**** THE AGENT STATE IS THE ENERGY **** -0.0354111
Set reward :  2.001932501693503
****CALL STEP****
Action chosen at step:  [[54.86395  66.96789  67.72156 ]
 [55.090843 56.86407  24.38909 ]
 [54.741135 48.173786 41.809074]
 [63.704277 68.34616  96.26047 ]
 [58.60193  49.091362 68.49833 ]
 [31.219677 42.734642 20.887665]
 [38.9843   40.21889  53.785553]
 [64.81041  50.736244 59.754433]
 [58.9299   52.973225 50.39812 ]
 [62.815144 64.275826 53.312687]
 [48.243042 90.559715 76.79494 ]
 [77.51802  67.322754 47.04638 ]
 [46.715733 73.9226   72.72354 ]
 [47.232536 32.981674 77.51976 ]
 [ 7.033104 75.997925 23.78005 ]
 [63.01934  87.206345 66.17937 ]
 [68.4085   39.923317 53.83554 ]
 [81.34435  72.56828  48.93481 ]
 [28.934284 81.47498  51.862186]
 [33.533363 48.596992 47.617798]
 [56.82255  71.806305 65.44697 ]
 [60.90691  41.853146 66.76821 ]
 [77.42

With this action the energy is:  -0.0588025
With this action the full dim is:  50  and princip dim is:  49
#### THE ACTION IS A GOOD ONE ####
**** THE AGENT STATE IS THE ENERGY **** -0.0588025
Set reward :  3.628557910596414
****CALL STEP****
Action chosen at step:  [[ 63.240643   61.68389    79.283356 ]
 [ 31.142334   91.173706   50.80047  ]
 [ 54.37181    52.619247   39.784447 ]
 [ 45.566154   53.149208   87.19956  ]
 [  1.         56.09098    64.87116  ]
 [ 32.00168    57.366398   31.99883  ]
 [ 49.309628   63.804226   83.08723  ]
 [ 53.41204    80.46756    84.25429  ]
 [ 46.2168     57.648174   34.55806  ]
 [109.81134    31.807705   69.43685  ]
 [ 37.694237   90.32414    90.08254  ]
 [ 82.02277    49.127056   69.44205  ]
 [ 28.243864   64.920494   85.749115 ]
 [ 39.28627    69.3832     62.769047 ]
 [ 14.034622   43.29528    35.251163 ]
 [ 39.180367   56.79632    46.038017 ]
 [ 91.63393    39.25923    14.613132 ]
 [ 72.96652    52.10573    50.77485  ]
 [ 50.155563   89.54036    21.0

With this action the energy is:  -0.0826126
With this action the full dim is:  50  and princip dim is:  50
#### THE ACTION IS A GOOD ONE ####
**** THE AGENT STATE IS THE ENERGY **** -0.0826126
Set reward :  5.284299492909538
Episode 8 ... Score: 46.213
*****CALL RESET******
Action chosen at reset:  [0.]
****CALL STEP****
Action chosen at step:  [[31.240824 59.935417 48.8548  ]
 [63.42421  38.227295 55.313545]
 [77.09405  72.68236  42.310722]
 [76.842705 55.721817 51.920025]
 [56.929226 33.61811  61.731754]
 [30.745642 41.47526  31.725473]
 [49.416897 59.704914 49.30264 ]
 [23.559704 60.945908 66.86305 ]
 [65.7872   69.709    30.432596]
 [39.46923  44.627266 52.554585]
 [54.86277  40.773296 80.75219 ]
 [83.41913  40.652508 57.3116  ]
 [52.14695  41.963264 78.52382 ]
 [85.8647   53.884666 55.78762 ]
 [62.922066 64.433495 61.964592]
 [43.202557 71.839294 43.757465]
 [67.48628  47.942253 72.907005]
 [63.37357  66.97316  50.979084]
 [57.22541  60.86277  45.590195]
 [41.900635 43.66946  40.0

With this action the energy is:  -0.0822789
With this action the full dim is:  50  and princip dim is:  49
#### THE ACTION IS A GOOD ONE ####
**** THE AGENT STATE IS THE ENERGY **** -0.0822789
Set reward :  5.261094173905507
****CALL STEP****
Action chosen at step:  [[ 62.214813  34.434113  11.584568]
 [ 51.268253  59.227818  47.97581 ]
 [ 82.98835  110.        35.6594  ]
 [ 96.943375  43.68708   56.017406]
 [ 69.93837   56.183823  62.09902 ]
 [ 86.85576   33.33281   58.46516 ]
 [ 62.135494  41.55043   77.247826]
 [ 24.918013  43.18961   61.602276]
 [ 57.55326   85.60373   50.43951 ]
 [ 83.026436  24.29991   71.16204 ]
 [ 46.546894  32.4787    71.78644 ]
 [ 68.35512   47.47901   28.155354]
 [ 69.559044  47.201637  76.55533 ]
 [ 82.7007    38.499878  56.277523]
 [ 28.049593  85.61439   62.774147]
 [ 22.64835   79.95729   67.2451  ]
 [ 71.21335   29.428913  89.03184 ]
 [ 54.31028   80.3522    61.60347 ]
 [ 66.539566  29.029589  40.5934  ]
 [ 66.90608   54.53421   50.00857 ]
 [ 48.39028  

With this action the energy is:  -0.0703178
With this action the full dim is:  50  and princip dim is:  47
#### THE ACTION IS A GOOD ONE ####
**** THE AGENT STATE IS THE ENERGY **** -0.0703178
Set reward :  4.429325695814045
****CALL STEP****
Action chosen at step:  [[ 59.933907   67.993065   23.370197 ]
 [ 68.59425    59.474743   46.780914 ]
 [ 37.246777   76.05448    22.617554 ]
 [106.6116     40.932747   48.036194 ]
 [ 76.27219    84.36238    46.98077  ]
 [ 71.79177    23.54012    13.7167015]
 [ 69.502205   88.2005     58.76285  ]
 [ 60.66497    38.52665    57.160633 ]
 [ 74.78988   101.21445    66.68398  ]
 [ 61.776512   36.824642   43.955925 ]
 [ 35.358414   87.81642    83.82697  ]
 [ 88.338135   46.81646    74.97836  ]
 [ 72.99154   100.89432    36.399776 ]
 [ 71.87889    36.727997   55.960022 ]
 [ 44.701004   78.30735    71.026566 ]
 [ 38.394077   72.34662    51.67237  ]
 [ 49.80279     2.3840294  75.72258  ]
 [ 69.52249    53.932858   74.1877   ]
 [ 51.113792   44.170414   61.6

## From my `ppo_agent.py` code

In [ ]:
agent = PPO_agent(env, state_size, act_size, seed = 0)

In [ ]:
def run_ppo(max_t_step):
    
    traj_obs, traj_act, rew_t_fut, traj_log_pol, len_traj = agent.collect_traj()

## Random search as in original SVM

In [ ]:
state = env.reset()
scores = []
step = 0
score = 0.0

while True:
    print(".....STEP.....", step)
    action = env.action_space.sample()
    next_state, reward, done, info = env.step(action)
    step = step + 1
    score += reward
    scores.append(score)
    state = next_state
    if done:
        break